In [1]:
import pandas as pd

In [5]:
with open('./json/australian_user_reviews.json','r') as read:
    data = read.readlines()

data = [eval(i) for i in data]
df = pd.DataFrame(data)

In [7]:
df.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [ ]:
#aplanamos datos de nuestra columna reviews
df = pd.json_normalize(df['reviews'].explode()).reset_index(drop=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  object
 6   review       59305 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


usamos funcion para comporbar campos nulos y vacios de nuestra data

In [11]:
#funcion donde se comprueban valores nulos de las columnas
def analizar_datos(dataframe):
    num_filas, num_columnas = dataframe.shape
   
    resultados = {}
    
    for columna in dataframe.columns:
        nulos = dataframe[columna].isnull().sum()
        vacios = dataframe[columna].apply(lambda x: x == '').sum()
        
        resultados[columna] = {
            'filas_nulas': nulos,
            'campos_vacios': vacios
        }
    
    return resultados

In [12]:
resultados_analisis = analizar_datos(df) 
resultados_analisis

{'funny': {'filas_nulas': 28, 'campos_vacios': 51154},
 'posted': {'filas_nulas': 28, 'campos_vacios': 0},
 'last_edited': {'filas_nulas': 28, 'campos_vacios': 53165},
 'item_id': {'filas_nulas': 28, 'campos_vacios': 0},
 'helpful': {'filas_nulas': 28, 'campos_vacios': 0},
 'recommend': {'filas_nulas': 28, 'campos_vacios': 0},
 'review': {'filas_nulas': 28, 'campos_vacios': 30}}

se observa demasiada cantidad de camposvacios en funny y last-edited

porcederemos a trabajar la columna posted ya que presenta unos valores que serian un desafio trabjar querys con ciertos parametros, correremos una funcion que nos permita depurar la palabra postede de nuestras filas

In [15]:
#remplazamos valores y limipar espacios en blanco
def limpiar_posted(texto):
    if isinstance(texto, str):
        texto_limpio = texto.replace('Posted', '').replace(',', '').replace('.', '').strip()
        return texto_limpio
    return texto

In [16]:
df['posted'] = df['posted'].apply(limpiar_posted)

se ve mucho mas claro pero hay varias filas con campos sin la fecha completa, se va realizar un analisis estadistico para poder asignar una fecha a esos campos y no eliminar data

In [33]:
df.describe()

,funny,posted,last_edited,item_id,helpful,recommend,review
count,59305,59305,59305,59305,59305,59305,59305
unique,186,1906,1015,3682,1344,2,55313
top,,June 21 2014,,730,No ratings yet,True,good game
freq,51154,225,53165,3759,30168,52473,100


en el top se que el 2014 es la fecha mas frecuente por lo que tomo la decision de asignarle esa fecha a los campos que no la posean para seguir conservanfo info en vez de eliminar

In [71]:
#convertimos a datime la columna  creada para asignarle el anio 2014
def extraer_fecha(fecha):
    try:
        if pd.isnull(fecha):
            return None, None, None
        fecha_dt = pd.to_datetime(str(fecha) + ' 2014', errors='raise')
        return fecha_dt.year, fecha_dt.month, fecha_dt.day
    except ValueError:
        try:
            fecha_dt = pd.to_datetime(str(fecha) + ' 2014', errors='coerce')
            return fecha_dt.year, fecha_dt.month, fecha_dt.day
        except (ValueError, AttributeError):
            return None, None, None

df['year'], df['month'], df['day'] = zip(*df['posted'].map(extraer_fecha))

df['month'] = df['month'].astype(pd.Int64Dtype())
df['day'] = df['day'].astype(pd.Int64Dtype())
df['year'] = df['year'].astype(pd.Int64Dtype())

In [72]:
anio_mas_frecuente = df['year'].mode()[0]
df['year'].fillna(anio_mas_frecuente, inplace=True)

In [76]:
df.isnull().sum()

funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
year            0
month          53
day            53
dtype: int64

ahora procedemos a unificar la info recolpilada en una sola columna y depurar las columnas creadas anteriormente, son my pocos datos nulos, viendo la canmtidad a proceder eliminarlos no afectaran nuestra data

In [77]:
#transformaos la nueva columna y procedemos a eliminar las creadas anteriormente
df['new_posted'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
df.drop(columns=['year', 'month', 'day','posted'], inplace=True)

- 'funny': {'filas_nulas': 28, 'campos_vacios': 51154},
- 'last_edited': {'filas_nulas': 28, 'campos_vacios': 53165}

al poseer tantos campos vacios y no tener mucha info para asignarle a estos campos se procede a depurar estas columnas y transformamos nuestra columna id para que sea mas facil trabajar consultas con id tipo integer

In [105]:
df.drop(columns=['funny', 'last_edited'], inplace=True)

In [ ]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [5]:
df['item_id'] = df['item_id'].astype(str)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59280 entries, 0 to 59332
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   item_id             59280 non-null  object        
 1   helpful             59280 non-null  object        
 2   recommend           59280 non-null  bool          
 3   review              59280 non-null  object        
 4   new_posted          59280 non-null  datetime64[ns]
 5   sentiment_analysis  59280 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(1), object(3)
memory usage: 2.8+ MB


empezaremos a trabajar la columna review ya que poseen muchs campos con caracteres no deseados como emojis, con una funcion que nos permitar instanciar nuestras columnas

In [133]:
import re

def limpiar_review(texto):
    texto_limpio = texto.replace("'", "").replace('"', '')
    texto_limpio = texto_limpio.lower().capitalize()
    texto_limpio = re.sub(r'[^a-zA-Z\s.,!?]', '', texto_limpio)
    return texto_limpio

In [134]:
df['review'] = df['review'].apply(limpiar_review)

verificamos nuestros campos con nuestra funcion anterior

In [164]:
resultados_analisis = analizar_datos(df) 
resultados_analisis

{'item_id': {'filas_nulas': 0, 'campos_vacios': 0},
 'helpful': {'filas_nulas': 0, 'campos_vacios': 0},
 'recommend': {'filas_nulas': 0, 'campos_vacios': 0},
 'review': {'filas_nulas': 0, 'campos_vacios': 773},
 'new_posted': {'filas_nulas': 0, 'campos_vacios': 0},
 'sentiment_analysis': {'filas_nulas': 0, 'campos_vacios': 0}}

hay 773 campos vacios en nuestra columna reviews pero no transformaremos esos campos, nos guiaremos con nuestra columna recommend par seguir nuestras quuery, y empezaremos a realizar nuestro modelo de entranamiento para nuestra columna sentiment-analysis

In [167]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/bemontx/nltk_data...


True

instanciamos nuestro modelo

In [168]:
sia = SentimentIntensityAnalyzer()

creamos una funcion 

In [172]:
def analizar_sentimiento(texto):
    if not texto:  # Si no hay texto presente, devolver neutral (1)
        return 1
    sentiment_score = sia.polarity_scores(texto)['compound']
    if sentiment_score > 0.05:  # Si el sentimiento es positivo
        return 2
    elif -0.05 <= sentiment_score <= 0.05:  # Si el sentimiento es neutral
        return 1
    else:  # Si el sentimiento es negativo
        return 0

df['sentiment_analysis'] = df['review'].apply(analizar_sentimiento)

valores en la columna sentiment_analysis
- 0 negativo
- 1 neutral
- 2 positivo

In [16]:
df.sample(5)

,item_id,helpful,recommend,review,new_posted,sentiment_analysis
22069,263320,No ratings yet,True,Koolzorthis game is so good and yeah. it is ju...,2015-03-14,2
25644,4000,1 of 1 people (100%) found this review helpful,True,This is most likely one of the greatest sandbo...,2014-10-18,2
21756,440,No ratings yet,True,Its the greatest hat simulator of all time! to...,2015-02-18,2
11055,17500,No ratings yet,True,Es un juego muy bueno quizas no tenga los mejo...,2014-05-04,0
17002,384021,2 of 5 people (40%) found this review helpful,True,Made a cloaker puke.take that loser! haha!you ...,2015-07-17,0


In [2]:
df = pd.read_parquet('./parquet/australian_user_reviews.parquet')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59280 entries, 0 to 59332
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   item_id             59280 non-null  object        
 1   helpful             59280 non-null  object        
 2   recommend           59280 non-null  bool          
 3   review              59280 non-null  object        
 4   new_posted          59280 non-null  datetime64[ns]
 5   sentiment_analysis  59280 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(1), object(3)
memory usage: 2.8+ MB


In [10]:
# df.to_parquet('australian_user_reviews.parquet')